In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from scipy.io import netcdf
from scipy import stats
import random
import os
import torch
from statsmodels.tsa.statespace.varmax import VARMAX
import tensorflow as tf
from torch.utils.data import Dataset
from transformers import PatchTSTConfig, PatchTSTForPrediction, AutoformerModel, AutoformerConfig, AutoformerForPrediction, TimeSeriesTransformerConfig, TimeSeriesTransformerForPrediction, InformerConfig, InformerForPrediction, is_torch_xla_available
from transformers.optimization import Adafactor, get_cosine_schedule_with_warmup, AdamW
from transformers import Trainer, TrainingArguments
import gc
import time
import logging
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Set logging level to error
logging.getLogger("transformers").setLevel(logging.ERROR)

2024-06-30 04:51:52.663630: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-30 04:51:52.663737: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-30 04:51:52.797597: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
class MainConfig:
    context_length = 10
    lags_sequence = [1, 2, 3, 4, 5, 6, 7]
    prediction_length = 14
    wandb = False
    train_batch_size = 4
    valid_batch_size = 8
    lr = 1e-4
    epochs = 20
    device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
YEARS = list(range(2000, 2025))

In [4]:
if MainConfig.wandb:
    import wandb
    os.environ["WANDB_DISABLED"] = "false"
    os.environ["WANDB_RUN_GROUP"] = "experiment-" + wandb.util.generate_id()
    user_secrets = UserSecretsClient()
    secret_value_0 = user_secrets.get_secret("wandb_api")
    wandb.login(key=secret_value_0)
    wandb.init(
        project="science-llm-mc",
        group="experiment-2",
        job_type=f"fold_{fold}"
    )
else:
    os.environ["WANDB_DISABLED"] = "true"

In [5]:
def seedBasic(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
def seedTorch(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
      
def seedEverything(seed):
    seedBasic(seed)
    seedTorch(seed)
    
seedEverything(2007)

In [6]:
def build_dataset():
    data = []
    for year in YEARS:
        file2read = netcdf.NetCDFFile(f"/kaggle/input/airdataset/air.{year}.nc",'r')
        variables = file2read.variables.keys()
        data.append({k : file2read.variables[k][:]*1 for k in variables})
        data[-1]["air"] = np.abs(data[-1]["air"])
        
    return data

In [7]:
def process_location(location, data):
    dataset = []
    
    for j, yearly_data in enumerate(data):
        dataset.append(yearly_data["air"][:,:,location[0], location[1]])
        
    dataset = np.concatenate(dataset, 0)
    
 #   dataset = pd.DataFrame(dataset / 10000)
    
#    dataset.columns = [f"Level {i+1}" for i in range(12)]
    
    return dataset, get_dates(len(dataset))

In [8]:
def get_dates(n):
    today = datetime.now()

    dates_list = [(today - timedelta(days=i-1)) for i in range(n)]
    dates_processed = [[date.month, date.day, date.year] for date in dates_list]
    return np.array(dates_processed[::-1])

In [9]:
def get_dates_ahead(n):
    today = datetime.now()

    dates_list = [(today + timedelta(days=i)) for i in range(n)]
    dates_processed = [[date.month, date.day, date.year] for date in dates_list]
    return np.array(dates_processed)

In [10]:
class WeatherDataset(Dataset):
    def __init__(self, dict_values):
        self.dict_values = dict_values
        
    def __len__(self):
        return len(self.dict_values["past_values"])
    
    def __getitem__(self, idx):
        return {
            k : torch.tensor(v[idx]).float() for k, v in self.dict_values.items() 
        }

In [11]:
def run(location_idxs, data):
        
    processed_data, dates = process_location(location_idxs, data)
        
    dict_vals = {
        "past_values" : [],
        "past_time_features" : [],
        "past_observed_mask" : [],
        "future_values" : [],
        "future_time_features" : [],
    }

    for i in range(0, len(processed_data) - MainConfig.context_length - MainConfig.prediction_length - max(MainConfig.lags_sequence), MainConfig.context_length + MainConfig.prediction_length + max(MainConfig.lags_sequence)):
        dict_vals["past_values"].append(processed_data[i : i + MainConfig.context_length + max(MainConfig.lags_sequence)])
        dict_vals["past_time_features"].append(dates[i : i + MainConfig.context_length + max(MainConfig.lags_sequence)])
        dict_vals["past_observed_mask"].append(np.ones((MainConfig.context_length + max(MainConfig.lags_sequence), 12)))

        dict_vals["future_values"].append(processed_data[i + MainConfig.context_length + max(MainConfig.lags_sequence) : i + MainConfig.context_length + max(MainConfig.lags_sequence) + MainConfig.prediction_length])
        dict_vals["future_time_features"].append(dates[i + MainConfig.context_length + max(MainConfig.lags_sequence) : i + MainConfig.context_length + max(MainConfig.lags_sequence) + MainConfig.prediction_length])
        
    train_ds = WeatherDataset(dict_vals)
        
    config = TimeSeriesTransformerConfig(
                          prediction_length=MainConfig.prediction_length, 
                          context_length=MainConfig.context_length,
                          lags_sequence=MainConfig.lags_sequence,
                          num_time_features=3,                              
                          d_model=128,
                          encoder_layers=2,
                          decoder_layers=2,
                          encoder_ffn_dim=64,
                          decoder_ffn_dim=64,
                          distribution_output="student_t",
                          moving_average=35,
                          num_parallel_samples=100,
                          input_size=12,                
        )
    model = TimeSeriesTransformerForPrediction(config).to(MainConfig.device)
    
    optimizer = AdamW(model.parameters(), lr=MainConfig.lr)
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=200, num_training_steps=int(len(train_ds)/MainConfig.train_batch_size)*MainConfig.epochs)

    arguments = TrainingArguments(output_dir="/kaggle/working/", 
                          learning_rate=MainConfig.lr, 
                          per_device_train_batch_size=MainConfig.train_batch_size, 
                          per_device_eval_batch_size=MainConfig.valid_batch_size, 
                          save_strategy="no",
                          num_train_epochs=10,
                          report_to="wandb" if MainConfig.wandb else None,
                          ignore_data_skip=True,
                          warmup_ratio=0.8,
                          label_smoothing_factor=0.0,
                          logging_strategy='no',
                          )

    trainer = Trainer(model=model, 
             train_dataset=train_ds, 
             args=arguments,
             optimizers=(optimizer, scheduler)
             )

    trainer.train()
    
    past_values = processed_data[-(MainConfig.context_length + max(MainConfig.lags_sequence)):]
    past_time_features = dates[-(MainConfig.context_length + max(MainConfig.lags_sequence)):]
    past_observed_mask = np.ones((MainConfig.context_length + max(MainConfig.lags_sequence), 12))

    model.eval()
    
    sample = {
        "past_values" : past_values,
        "past_time_features" : past_time_features,
        "past_observed_mask" : past_observed_mask,
        "future_time_features" : get_dates_ahead(100),
    }

    sample = {k : torch.tensor(v).unsqueeze(0).float().to(MainConfig.device) for k, v in sample.items()}
    
    outputs = model.generate(
            past_values = sample["past_values"],
            past_time_features = sample["past_time_features"],
            past_observed_mask = sample["past_observed_mask"],
            future_time_features = sample["future_time_features"],
        )
    
    preds = stats.trim_mean(outputs.sequences[0].cpu().detach().numpy(), 0.25, axis=0)
    
    del model, trainer, optimizer, scheduler, dict_vals
    torch.cuda.empty_cache()
    gc.collect()
    
    return np.asarray(preds * -10000, dtype="int")

In [12]:
data = build_dataset()

/tmp/ipykernel_25/1538903851.py:4: DeprecationWarning: Please use `NetCDFFile` from the `scipy.io` namespace, the `scipy.io.netcdf` namespace is deprecated.
  file2read = netcdf.NetCDFFile(f"/kaggle/input/airdataset/air.{year}.nc",'r')
/opt/conda/lib/python3.10/site-packages/scipy/io/_netcdf.py:305: RuntimeWarning: Cannot close a netcdf_file opened with mmap=True, when netcdf_variables or arrays referring to its data still exist. All data arrays obtained from such files refer directly to data on disk, and must be copied before the file can be cleanly closed. (See netcdf_file docstring for more information on mmap.)
  warnings.warn((


In [13]:
lat = [data[0]["lat"][0:34][i] for i in range(34) if i % 3 == 0]
lon = [data[0]["lon"][10:70][i] for i in range(60) if i % 3 == 0]

In [14]:
def MultiExponentialSmoothing(data):
    results = []
    for level in range(12):
        model = ExponentialSmoothing(processed_data[:,level], seasonal='add', seasonal_periods=365)
        model_fit = model.fit()
        n_forecast = 365
        forecast = model_fit.forecast(steps=n_forecast)
        results.append(forecast.reshape(365, 1))
        
    return np.concatenate(results, 1)

In [15]:
cur = 0
total = len(lat) * len(lon)

results = {}

for i in range(0, len(lat)):
    for j in range(0, len(lon)):
        start_time = time.time()

        processed_data, dates = process_location((i, j), data)
        
        output = MultiExponentialSmoothing(processed_data)
        
        results[f"{lat[i]}_{lon[j]}"] = output
        
        cur += 1

        print(f"{cur} / {total}")
        
        end_time = time.time()
        
        elapsed_time = end_time - start_time
        print(f'Elapsed time: {elapsed_time} seconds')
        print(f'Estimated Time: {(total - cur) * elapsed_time} seconds')

/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_

1 / 240
Elapsed time: 4.963141679763794 seconds
Estimated Time: 1186.1908614635468 seconds


/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_

2 / 240
Elapsed time: 5.047758340835571 seconds
Estimated Time: 1201.366485118866 seconds


/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_

...

/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_

239 / 240
Elapsed time: 4.713611841201782 seconds
Estimated Time: 4.713611841201782 seconds


/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_

240 / 240
Elapsed time: 4.6491711139678955 seconds
Estimated Time: 0.0 seconds


/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


In [16]:
import json

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

with open('air_temps.json', 'w') as f:
    json.dump(results, f, cls=NpEncoder)